In [3]:
import whisper
import time
import difflib
from IPython.display import display, HTML
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
model_sizes = ["tiny", "base", "small", "medium", "large"]
dic_transcricoes = {}

for model_size in model_sizes:
    start_time = time.time()  # início da iteração

    model = whisper.load_model(model_size)
    print(f"\nCarregado modelo: {model_size}, dims: {model.dims}")

    result = model.transcribe("../DataFrames/entrevista-pedro-calais-98news-26-06-2025.mp3", fp16=False)
    dic_transcricoes[model_size] = result["text"]

    end_time = time.time()  # fim da iteração
    elapsed_time = end_time - start_time
    print(f"⏱ Tempo de execução ({model_size}): {elapsed_time:.2f} segundos")


Carregado modelo: tiny, dims: ModelDimensions(n_mels=80, n_audio_ctx=1500, n_audio_state=384, n_audio_head=6, n_audio_layer=4, n_vocab=51865, n_text_ctx=448, n_text_state=384, n_text_head=6, n_text_layer=4)
⏱ Tempo de execução (tiny): 15.20 segundos

Carregado modelo: base, dims: ModelDimensions(n_mels=80, n_audio_ctx=1500, n_audio_state=512, n_audio_head=8, n_audio_layer=6, n_vocab=51865, n_text_ctx=448, n_text_state=512, n_text_head=8, n_text_layer=6)
⏱ Tempo de execução (base): 23.41 segundos

Carregado modelo: small, dims: ModelDimensions(n_mels=80, n_audio_ctx=1500, n_audio_state=768, n_audio_head=12, n_audio_layer=12, n_vocab=51865, n_text_ctx=448, n_text_state=768, n_text_head=12, n_text_layer=12)
⏱ Tempo de execução (small): 57.65 segundos

Carregado modelo: medium, dims: ModelDimensions(n_mels=80, n_audio_ctx=1500, n_audio_state=1024, n_audio_head=16, n_audio_layer=24, n_vocab=51865, n_text_ctx=448, n_text_state=1024, n_text_head=16, n_text_layer=24)
⏱ Tempo de execução (medi

In [14]:
def exibe_diferenca_html_dark(texto1, texto2):
    
    palavras1 = texto1.split()
    palavras2 = texto2.split()

    diff = difflib.ndiff(palavras1, palavras2)
    resultado = []

    for token in diff:
        if token.startswith('  '):  # palavras iguais
            resultado.append(token[2:])
        elif token.startswith('- '):  # removidas
            resultado.append(f'<span style="color:#ff5555;">{token[2:]}</span>')  # vermelho
        elif token.startswith('+ '):  # adicionadas
            resultado.append(f'<span style="color:#a6e22e;">{token[2:]}</span>')  # verde-limão

    html_text = ' '.join(resultado)
    display(HTML(html_text))

In [15]:
exibe_diferenca_html_dark(dic_transcricoes['tiny'], dic_transcricoes['large'])

In [7]:
stop_pt = ['de','a','o','que','e','do','da','em','um','para','com','não','uma', 'os','no','se','na','por','mais']
vectorizer = TfidfVectorizer(stop_words=stop_pt)
tfidf_matrix = vectorizer.fit_transform(dic_transcricoes.values())

# --- Similaridade de cosseno ---
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# --- Matriz de correlação em DataFrame ---
model_sizes = list(dic_transcricoes.keys())
matriz_correlacao = pd.DataFrame(cosine_sim, index=model_sizes, columns=model_sizes)

print("\nMatriz de Correlação (similaridade de cosseno):")
print(matriz_correlacao.round(3))


Matriz de Correlação (similaridade de cosseno):
         tiny   base  small  medium  large
tiny    1.000  0.939  0.942   0.934  0.901
base    0.939  1.000  0.972   0.976  0.923
small   0.942  0.972  1.000   0.983  0.947
medium  0.934  0.976  0.983   1.000  0.944
large   0.901  0.923  0.947   0.944  1.000
